In [ ]:
#### To Show the Points on both original and transformed images ######

import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import os
import datetime


# Read correspondances
pts_original_affine = np.loadtxt(os.path.join('PartA','correspondances','perspective','lena.csv'), delimiter=',', skiprows=1).astype(np.float32)

pts_original = pts_original_affine[:, 0:2]
pts_affine = pts_original_affine[:, 2:4]

# Read images
img_original = cv.imread(os.path.join('PartA','original','lena.png'))
img_affine = cv.imread(os.path.join('PartA','perspective','lena.png'))

# BGR to RGB
img_original = cv.cvtColor(img_original, cv.COLOR_BGR2RGB)
img_affine = cv.cvtColor(img_affine, cv.COLOR_BGR2RGB)

# Display original image
plt.scatter(x=pts_original[:, 0], y=pts_original[:, 1], c='r', s=40)
plt.imshow(img_original)
plt.show()
plt.clf()

# Display affine image
plt.scatter(x=pts_affine[:, 0], y=pts_affine[:, 1], c='r', s=40)
plt.imshow(img_affine)
plt.show()
plt.clf()


# Mathematical Functions

In [2]:
###### Some Mathematical Functions ###########

def pseudo_inverse(A):
	r , c = A.shape
	U, S, Vh = np.linalg.svd(A, full_matrices=True, compute_uv=True)
	n = len(S)
	S = np.diag(S)
	nonzero_mask = S != 0
	S_ = np.copy(S)
	S_[nonzero_mask] = 1/S[nonzero_mask]
	Uh = np.transpose(U)
	V = np.transpose(Vh)
	smat = np.zeros((c,r))
	smat[ :n , :n] = S_
	return np.dot(V , np.dot(smat , Uh))


def solve_equation(A,b):
	A_ = pseudo_inverse(A)
	return np.dot(A_,b)

def calculate_error(actual, expected):
	return np.mean((actual - expected)**2)

def order_corners(pts):
	assert(len(pts) == 4)

	pts_rev = [(y,x) for x,y in pts]
	pts_rev.sort()
	ord_pts = [(x,y) for y,x in pts_rev]

	if ord_pts[0][0] > ord_pts[1][0]:
		ord_pts[0] , ord_pts[1] = ord_pts[1], ord_pts[0]

	if ord_pts[2][0] < ord_pts[3][0]:
		ord_pts[2] , ord_pts[3] = ord_pts[3], ord_pts[2]

	return np.array(ord_pts)

# Part - A

In [3]:
###### Part A Functionalities ##################

def get_affine_transform_method_a(original_points, affine_points):
	random_indices = np.random.choice(original_points.shape[0], size=3, replace=False)
	original_points = np.array(original_points[random_indices,:])
	affine_points = np.array(affine_points[random_indices,:])

	original_points = np.hstack((original_points,np.ones((3,1))))

	A , b = np.zeros((6,6)) , np.zeros((6,1))
	for i in range(3):
		A[2*i,:3] = np.array(original_points[i,:])
		A[2*i+1,3:] = np.array(original_points[i,:])
        
		b[2*i:2*i+2,0] = np.array(affine_points[i,:])
    
	mat = np.zeros((2,3))
	ans = solve_equation(A,b)
	mat[0,:] = ans[:3,0]
	mat[1,:] = ans[3:,0]
                
	return mat

def get_affine_transform_method_b(original_points, affine_points):
	n = original_points.shape[0]
	original_points = np.array(np.hstack((original_points,np.ones((n,1)))))
	affine_points = np.array(affine_points)

	A , b = np.zeros((2*n,6)) , np.zeros((2*n,1))
	for i in range(n):
		A[2*i,:3] = np.array(original_points[i,:])
		A[2*i+1,3:] = np.array(original_points[i,:])
        
		b[2*i:2*i+2,0] = np.array(affine_points[i,:])
    
	mat = np.zeros((2,3))
	ans = solve_equation(A,b)
	mat[0,:] = ans[:3,0]
	mat[1,:] = ans[3:,0]
                
	return mat
	
def warp_affine(img, M):
    R,C,_ = img.shape
    dst = np.zeros((R,C,img.shape[2]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            j_ , i_ = np.dot(M, [[j],[i],[1]]).astype(int)
            if i_ >= 0 and i_ < R and j_ >= 0 and j_ < C:
                dst[i_,j_] = img[i,j]
            
    return dst

def get_perspective_transform_method_a(original_points, perspective_points):
	random_indices = np.random.choice(original_points.shape[0], size=4, replace=False)
	original_points = np.array(original_points[random_indices,:])
	perspective_points = np.array(perspective_points[random_indices,:])

	original_points = np.hstack((original_points,np.ones((4,1))))

	A , b = np.zeros((8,8)) , np.zeros((8,1))
	A[:4,:3] = np.array(original_points)
	A[4:,3:6] = np.array(original_points)

	b[:4,0] = np.array(perspective_points[:,0])
	b[4:,0] = np.array(perspective_points[:,1])

	A[:4,6:] = np.array(-1*b[:4,:]*original_points[:,:2])
	A[4:,6:] = np.array(-1*b[4:,:]*original_points[:,:2])
    
	mat = np.ones((3,3))
	ans = solve_equation(A,b)
	mat[0,:] = ans[:3,0]
	mat[1,:] = ans[3:6,0]
	mat[2,:2] = ans[6:,0]
                
	return mat

def get_perspective_transform_method_b(original_points, perspective_points):
	n = original_points.shape[0]
	original_points = np.array(np.hstack((original_points,np.ones((n,1)))))
	perspective_points = np.array(perspective_points)

	A , b = np.zeros((2*n,8)) , np.zeros((2*n,1))
	A[:n,:3] = np.array(original_points)
	A[n:,3:6] = np.array(original_points)

	b[:n,0] = np.array(perspective_points[:,0])
	b[n:,0] = np.array(perspective_points[:,1])

	A[:n,6:] = np.array(-1*b[:n,:]*original_points[:,:2])
	A[n:,6:] = np.array(-1*b[n:,:]*original_points[:,:2])
    
	mat = np.ones((3,3))
	ans = solve_equation(A,b)
	mat[0,:] = ans[:3,0]
	mat[1,:] = ans[3:6,0]
	mat[2,:2] = ans[6:,0]

	return mat

def warp_perspective(img, M):
    R,C,_ = img.shape
    dst = np.zeros((R,C,img.shape[2]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            dotP = np.dot(M, [[j],[i],[1]])
            j_ , i_ , _ = (dotP/dotP[2]).astype(int)
            if i_ >= 0 and i_ < R and j_ >= 0 and j_ < C:
                dst[i_,j_] = img[i,j]
            
    return dst

## Test part-A 

In [4]:
'''
This is a sample source code file.
It is not complete and does not work.
It just gives you an overall idea.
'''

org_base_dir = os.path.join('PartA','original')
files = os.listdir(org_base_dir)
# files = [os.path.join(org_base_dir,file_name) for file_name in files]

out_file = os.path.join('PartA','outputs',"result.txt")
fout = open(out_file,"w")
fout.write("Date-Time : "+str(datetime.datetime.now().strftime("%d-%m-%Y-%H:%M:%S"))+"\n")

def printf(msg,msg2 = None, msg3 = None):
    msg += str(msg2 if msg2 is not None else "") + str(msg3 if msg3 is not None else "")
    print(msg)
    fout.write(msg+"\n")

for file_name in files: # Loop through images
    original_image = cv.imread(os.path.join(org_base_dir,file_name))
    h , w , c = original_image.shape
    # print(original_image.shape)

     # Read the points
    csv_filename = file_name.rsplit('.', maxsplit=1)[0] + '.csv'
    affine_points_all = np.loadtxt(os.path.join('PartA','correspondances','affine',csv_filename), delimiter=',', skiprows=1).astype(np.float32)
    original_points_aff = np.array(affine_points_all[:,0:2])
    affine_points = np.array(affine_points_all[:,2:4])

    pers_points_all = np.loadtxt(os.path.join('PartA','correspondances','perspective',csv_filename), delimiter=',', skiprows=1).astype(np.float32)
    original_points_pers = np.array(pers_points_all[:,0:2])
    perspective_points = np.array(pers_points_all[:,2:4])

    # Use the built-in OpenCV functions
    random_affine_indices = np.random.choice(affine_points.shape[0], size=3, replace=False)
    affine_transform_builtin = cv.getAffineTransform(original_points_aff[random_affine_indices,:], affine_points[random_affine_indices,:])
    affine_image_builtin = cv.warpAffine(np.copy(original_image),affine_transform_builtin,(w,h),np.zeros(original_image.shape))

    random_pers_indices = np.random.choice(perspective_points.shape[0], size=4, replace=False)
    perspective_transform_builtin , _  = cv.findHomography(original_points_pers,perspective_points)
    pers_image_builtin = cv.warpPerspective(np.copy(original_image),perspective_transform_builtin,(w,h),flags=cv.INTER_LINEAR)


    # Use your own implemented functions
    affine_transform_method_a = get_affine_transform_method_a(original_points_aff.copy(), affine_points.copy())
    # affine_image_method_a = warp_affine(np.copy(original_image), affine_transform_method_a)
    affine_image_method_a = cv.warpAffine(np.copy(original_image), affine_transform_method_a,(w,h))

    affine_transform_method_b = get_affine_transform_method_b(original_points_aff.copy(), affine_points.copy())
    # affine_image_method_b = warp_affine(np.copy(original_image), affine_transform_method_b)
    affine_image_method_b = cv.warpAffine(np.copy(original_image), affine_transform_method_b,(w,h))

    perspective_transform_method_a = get_perspective_transform_method_a(original_points_pers[random_pers_indices,:],perspective_points[random_pers_indices,:])
    # perspective_image_method_a = warp_perspective(np.copy(original_image), perspective_transform_method_a)
    perspective_image_method_a = cv.warpPerspective(np.copy(original_image), perspective_transform_method_a,(w,h),flags=cv.INTER_LINEAR)

    perspective_transform_method_b = get_perspective_transform_method_b(original_points_pers.copy(), perspective_points.copy())
    # perspective_image_method_b = warp_perspective(np.copy(original_image), perspective_transform_method_b)
    perspective_image_method_b = cv.warpPerspective(np.copy(original_image), perspective_transform_method_b,(w,h),flags=cv.INTER_LINEAR)

    affine_error_method_a = calculate_error(affine_transform_method_a, affine_transform_builtin)
    affine_error_method_b = calculate_error(affine_transform_method_b, affine_transform_builtin)
    perspective_error_method_a = calculate_error(perspective_transform_method_a, perspective_transform_builtin)
    perspective_error_method_b = calculate_error(perspective_transform_method_b, perspective_transform_builtin)


    printf('############################################################')
    printf('Processed image:', file_name)
    printf('------------------------------------------------------------')
    printf('Affine transformation matrix, method A\n', affine_transform_method_a)
    printf('------------------------------------------------------------')
    printf('Affine transformation matrix, method B\n', affine_transform_method_b)
    printf('------------------------------------------------------------')
    printf('Perspective transformation matrix, method A\n', perspective_transform_method_a)
    printf('------------------------------------------------------------')
    printf('Perspective transformation matrix, method B\n', perspective_transform_method_b)
    printf('------------------------------------------------------------')
    printf('Error between my affine vs. built-in function, method A\n', affine_error_method_a)
    printf('------------------------------------------------------------')
    printf('Error between my affine vs. built-in function, method B\n', affine_error_method_b)
    printf('------------------------------------------------------------')
    printf('Error between my perspective vs. built-in function, method A\n', perspective_error_method_a)
    printf('------------------------------------------------------------')
    printf('Error between my perspective vs. built-in function, method B\n', perspective_error_method_b)
    printf('------------------------------------------------------------\n\n')


    # save generated images to ./outputs folder
    out_dir = os.path.join('PartA','outputs')
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    if not os.path.exists(os.path.join(out_dir,'affine')):
        os.mkdir(os.path.join(out_dir,'affine'))
    if not os.path.exists(os.path.join(out_dir,'perspective')):
        os.mkdir(os.path.join(out_dir,'perspective'))
    # os.chdir(out_dir)

    cv.imwrite(os.path.join(out_dir,'affine',file_name.rsplit('.', maxsplit=1)[0] + '_affine_method_a.png'), affine_image_method_a)
    cv.imwrite(os.path.join(out_dir,'affine',file_name.rsplit('.', maxsplit=1)[0] + '_affine_method_b.png'), affine_image_method_b)
    cv.imwrite(os.path.join(out_dir,'perspective',file_name.rsplit('.', maxsplit=1)[0] + '_perspective_method_a.png'), perspective_image_method_a)
    cv.imwrite(os.path.join(out_dir,'perspective',file_name.rsplit('.', maxsplit=1)[0] + '_perspective_method_b.png'), perspective_image_method_b)
    cv.imwrite(os.path.join(out_dir,'affine',file_name.rsplit('.', maxsplit=1)[0] + '_affine_builtin.png'), affine_image_builtin)
    cv.imwrite(os.path.join(out_dir,'perspective',file_name.rsplit('.', maxsplit=1)[0] + '_perspective_builtin.png'), pers_image_builtin)

fout.close()

############################################################
Processed image:lena.png
------------------------------------------------------------
Affine transformation matrix, method A
[[-1.00000000e+00 -3.38614809e-14  5.42000000e+02]
 [-2.92062354e-02 -1.01702204e+00  5.54905393e+02]]
------------------------------------------------------------
Affine transformation matrix, method B
[[-1.00000000e+00 -2.33381613e-16  5.42000000e+02]
 [-2.44098480e-03 -1.00198179e+00  5.42506973e+02]]
------------------------------------------------------------
Perspective transformation matrix, method A
[[ 1.42506654e-01  6.82001873e-01 -6.29940074e+01]
 [ 1.02051951e+00 -9.10734518e-01 -1.88656043e+01]
 [ 7.58956375e-04 -3.63502156e-03  1.00000000e+00]]
------------------------------------------------------------
Perspective transformation matrix, method B
[[ 1.25793397e-01  6.35218160e-01 -5.10545970e+01]
 [ 9.91427674e-01 -9.02435849e-01 -1.43639104e+01]
 [ 7.05585386e-04 -3.61050287e-03  1.00000

## part-A Additional

In [5]:
file_name = os.path.join('PartA','original',"computer.png")

original_image = cv.imread(file_name)
h , w , c = original_image.shape
# print(original_image.shape)

    # Read the points
csv_filename = "computer_add.csv"
affine_points_all = np.loadtxt(os.path.join('PartA','correspondances','affine',csv_filename), delimiter=',', skiprows=1).astype(np.float32)
pers_points_all = np.loadtxt(os.path.join('PartA','correspondances','perspective',csv_filename), delimiter=',', skiprows=1).astype(np.float32)
original_points_aff = np.array(affine_points_all[:,0:2])
affine_points = np.array(affine_points_all[:,2:4])

original_points_pers = np.array(pers_points_all[:,0:2])
perspective_points = np.array(pers_points_all[:,2:4])

# Use the built-in OpenCV functions
random_affine_indices = np.random.choice(affine_points.shape[0], size=3, replace=False)
affine_transform_builtin = cv.getAffineTransform(original_points_aff[random_affine_indices,:], affine_points[random_affine_indices,:])
affine_image_builtin = cv.warpAffine(np.copy(original_image),affine_transform_builtin,(w,h),np.zeros(original_image.shape))

random_pers_indices = np.random.choice(perspective_points.shape[0], size=4, replace=False)
perspective_transform_builtin , _  = cv.findHomography(original_points_pers,perspective_points)
pers_image_builtin = cv.warpPerspective(np.copy(original_image),perspective_transform_builtin,(w,h),flags=cv.INTER_LINEAR)


# Use your own implemented functions
affine_transform_method_a = get_affine_transform_method_a(original_points_aff.copy(), affine_points.copy())
# affine_image_method_a = warp_affine(np.copy(original_image), affine_transform_method_a)
affine_image_method_a = cv.warpAffine(np.copy(original_image), affine_transform_method_a,(w,h))

affine_transform_method_b = get_affine_transform_method_b(original_points_aff.copy(), affine_points.copy())
# affine_image_method_b = warp_affine(np.copy(original_image), affine_transform_method_b)
affine_image_method_b = cv.warpAffine(np.copy(original_image), affine_transform_method_b,(w,h))

perspective_transform_method_a = get_perspective_transform_method_a(original_points_pers[random_pers_indices,:],perspective_points[random_pers_indices,:])
# perspective_image_method_a = warp_perspective(np.copy(original_image), perspective_transform_method_a)
perspective_image_method_a = cv.warpPerspective(np.copy(original_image), perspective_transform_method_a,(w,h),flags=cv.INTER_LINEAR)

perspective_transform_method_b = get_perspective_transform_method_b(original_points_pers.copy(), perspective_points.copy())
# perspective_image_method_b = warp_perspective(np.copy(original_image), perspective_transform_method_b)
perspective_image_method_b = cv.warpPerspective(np.copy(original_image), perspective_transform_method_b,(w,h),flags=cv.INTER_LINEAR)

affine_error_method_a = calculate_error(affine_transform_method_a, affine_transform_builtin)
affine_error_method_b = calculate_error(affine_transform_method_b, affine_transform_builtin)
perspective_error_method_a = calculate_error(perspective_transform_method_a, perspective_transform_builtin)
perspective_error_method_b = calculate_error(perspective_transform_method_b, perspective_transform_builtin)


print('############################################################')
print('Processed image:', "coumputer.png")
print('------------------------------------------------------------')
print('Affine transformation matrix, method A\n', affine_transform_method_a)
print('------------------------------------------------------------')
print('Affine transformation matrix, method B\n', affine_transform_method_b)
print('------------------------------------------------------------')
print('Perspective transformation matrix, method A\n', perspective_transform_method_a)
print('------------------------------------------------------------')
print('Perspective transformation matrix, method B\n', perspective_transform_method_b)
print('------------------------------------------------------------')
print('Error between my affine vs. built-in function, method A\n', affine_error_method_a)
print('------------------------------------------------------------')
print('Error between my affine vs. built-in function, method B\n', affine_error_method_b)
print('------------------------------------------------------------')
print('Error between my perspective vs. built-in function, method A\n', perspective_error_method_a)
print('------------------------------------------------------------')
print('Error between my perspective vs. built-in function, method B\n', perspective_error_method_b)
print('------------------------------------------------------------')


# save generated images to ./outputs folder
out_dir = os.path.join('PartA','outputs')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
if not os.path.exists(os.path.join(out_dir,'affine')):
    os.mkdir(os.path.join(out_dir,'affine'))
if not os.path.exists(os.path.join(out_dir,'perspective')):
    os.mkdir(os.path.join(out_dir,'perspective'))
if not os.path.exists(os.path.join(out_dir,'affine','additional')):
    os.mkdir(os.path.join(out_dir,'affine','additional'))
if not os.path.exists(os.path.join(out_dir,'perspective','additional')):
    os.mkdir(os.path.join(out_dir,'perspective','additional'))
# os.chdir(out_dir)

cv.imwrite(os.path.join(out_dir,'affine','additional','computer_add_affine_method_a.png'), affine_image_method_a)
cv.imwrite(os.path.join(out_dir,'affine','additional','computer_add_affine_method_b.png'), affine_image_method_b)
cv.imwrite(os.path.join(out_dir,'perspective','additional','computer_add_perspective_method_a.png'), perspective_image_method_a)
cv.imwrite(os.path.join(out_dir,'perspective','additional','computer_add_perspective_method_b.png'), perspective_image_method_b)
cv.imwrite(os.path.join(out_dir,'affine','additional','computer_add_affine_builtin.png'), affine_image_builtin)
cv.imwrite(os.path.join(out_dir,'perspective','additional','computer_add_perspective_builtin.png'), pers_image_builtin)

############################################################
Processed image: coumputer.png
------------------------------------------------------------
Affine transformation matrix, method A
 [[   1.4271656     1.41262578 -433.87420966]
 [  -1.41277339    1.41311782  888.34893104]]
------------------------------------------------------------
Affine transformation matrix, method B
 [[   1.41867301    1.41845493 -431.30735603]
 [  -1.41305333    1.41342485  888.94135407]]
------------------------------------------------------------
Perspective transformation matrix, method A
 [[ 2.47215707e-01  1.14222941e-01  1.42738942e+01]
 [-7.54374645e-02  4.41652380e-01  3.50276266e+01]
 [-8.83221673e-04  1.47980368e-04  1.00000000e+00]]
------------------------------------------------------------
Perspective transformation matrix, method B
 [[ 1.76421321e-01  7.78619352e-02  4.22660270e+01]
 [-1.26385078e-01  3.66018495e-01  6.76228715e+01]
 [-9.40316369e-04  4.00701816e-05  1.00000000e+00]]
----

True

# Part B

In [6]:
##### part B Functionalities ###########

#################################################################

# Write a function Convolve (I, H). I is an image of varying size, H is a kernel of varying size.
# The output of the function should be the convolution result that is displayed.

def convolve(I, H):
	assert(H.shape[0] %2 == 1 and H.shape[1] %2 == 1)
	rows , cols = I.shape[0], I.shape[1]
	dtype = I.dtype
	dst = np.copy(I)
	v_pad = (H.shape[0]-1)//2
	h_pad = (H.shape[1]-1)//2
	for i in range(v_pad,rows-v_pad):
		for j in range(h_pad,cols-h_pad):
			conv = 0
			for a in range(H.shape[0]):
				for b in range(H.shape[1]):
					conv += H[a,b]*I[i-v_pad+a,j-h_pad+b]
			dst[i,j] = conv.astype(dtype)
    
	return dst

#################################################################

# Write a function Reduce(I) that takes image I as input and outputs a copy of the image resampled
# by half the width and height of the input. Remember to Gaussian filter the image before reducing it; 
# use separable 1D Gaussian kernels.

def reduce(I):
	rows , cols = I.shape[0], I.shape[1]
	dtype = I.dtype

	K_hor = np.array([[2,0,-1]]).astype(dtype)
	con_h = convolve(I,K_hor)

	K_ver = np.array([[2],[0],[-1]]).astype(dtype)
	con_v = convolve(I,K_ver)

	if len(I.shape) == 3:
		dst = np.zeros((rows//2,cols//2,I.shape[2]))
	else:
		dst = np.zeros((rows//2,cols//2))
	
	for i in range(0,rows,2):
		for j in range(0,cols,2):
			dst[i//2,j//2] = I[i,j].astype(dtype)

	return dst

#################################################################

# Write a function Expand(I) that takes image I as input and outputs a copy of the image expanded, 
# twice the width and height of the input.

def expand(I):
	rows , cols = I.shape[0], I.shape[1]
	dtype = I.dtype

	if len(I.shape) == 3:
		dst = np.zeros((rows*2,cols*2,I.shape[2]))
	else:
		dst = np.zeros((rows*2,cols*2))
	
	for i in range(rows-1):
		for j in range(cols-1):
			dst[2*i,2*j] = I[i,j].astype(dtype)
			dst[i*2+1,j*2] = (0.5*I[i,j]+0.5*I[i+1,j]).astype(dtype)
			dst[i*2,j*2+1] = (0.5*I[i,j]+0.5*I[i,j+1]).astype(dtype)
			dst[i*2+1,j*2+1] = (0.25*I[i,j]+0.25*I[i+1,j]+0.25*I[i,j+1]+0.25*I[i+1,j+1]).astype(dtype)

	for i in range(rows-1):
		dst[2*i,2*cols-2] = I[i,cols-1].astype(dtype)
		dst[i*2+1,cols*2-2] = (0.5*I[i,j]+0.5*I[i+1,cols-1]).astype(dtype)
		dst[i*2,cols*2-1] = I[i,cols-1].astype(dtype)
		dst[i*2+1,cols*2-1] = (0.5*I[i,cols-1]+0.5*I[i+1,cols-1]).astype(dtype)

	for j in range(cols-1):
		dst[2*rows-2,2*j] = I[rows-1,j].astype(dtype)
		dst[2*rows-1,j*2] = I[rows-1,j].astype(dtype)
		dst[2*rows-2,j*2+1] = (0.5*I[rows-1,j]+0.5*I[rows-1,j+1]).astype(dtype)
		dst[2*rows-1,j*2+1] = (0.5*I[rows-1,j]+0.5*I[rows-1,j+1]).astype(dtype)

	dst[2*rows-2,2*cols-2] = I[rows-1,cols-1].astype(dtype)
	dst[2*rows-1,2*cols-2] = I[rows-1,cols-1].astype(dtype)
	dst[2*rows-2,2*cols-1] = I[rows-1,cols-1].astype(dtype)
	dst[2*rows-1,2*cols-1] = I[rows-1,cols-1].astype(dtype)


	return dst

#################################################################

# Use the Reduce() function to write the GaussianPyramid(I,n) function, where n is the no. of levels.

def gaussianPyramid(I, n):
	if n == 0:
		return I
	
	K = (1/16)*np.array([[1,2,1],[2,4,2],[1,2,1]])
	conv = convolve(I,K)
	dst = reduce(conv).astype(I.dtype)
	
	return gaussianPyramid(dst,n-1)

#################################################################

# Use the above functions to write LaplacianPyramids(I,n) that produces n level Laplacian pyramid of I.

def laplacianPyramid(I, n):
	if n == 0:
		return [I]
	
	guss_i = gaussianPyramid(I,1)
	expnd = expand(guss_i)
	dst = (I - expnd).astype(I.dtype)
	
	return [dst] + laplacianPyramid(guss_i,n-1)

#################################################################

# Write the Reconstruct(LI,n) function which collapses the Laplacian pyramid LI of n levels 
# to generate the original image. Report the error in reconstruction using image difference.

def reconstruct(LI, n):
	if len(LI) <= 1:
		return LI[0]
	
	I = LI.pop(-1)
	dtype = I.dtype
	I_exp = expand(I)
	I_next = LI.pop(-1)
	dst = (I_next + I_exp).astype(dtype)
	LI.append(dst)

	return reconstruct(LI,n-1)

#################################################################

# blend function to blend two images using Laplace pyramid
# blend area of image is provided
def padding(I,dsize):
	r , c = I.shape[0], I.shape[1]
	srt_r, srt_c = dsize[0] // 2, dsize[1] // 2
	if len(I.shape) == 3:
		dst = np.zeros((r+dsize[0],c + dsize[1],I.shape[2]))
	else:
		dst = np.zeros((r+dsize[0],c + dsize[1]))
	dst[srt_r:srt_r+r,srt_c:srt_c+c] = I
	dst.astype(I.dtype)

	return dst

def blend_mask(c,cols,size = 64,upward = True):
	mask = np.zeros((1,cols,1))
	mask[0,:,0] = np.array([((i-c)/size + 0.5) if upward else ((c-i)/size + 0.5) for i in range(cols)])
	mask[mask >= 1] = 1
	mask[mask <= 0] = 0

	return mask

def blend_layer(L1,L2,c,size):
	assert(L1.shape == L2.shape)
	assert(L1.dtype == L2.dtype)
	mask1 = blend_mask(c,L1.shape[1],size,False)
	mask2 = blend_mask(c,L2.shape[1],size,True)

	dst = np.array(mask1 * L1 + mask2 * L2).astype(L1.dtype)

	return dst

def blend(I1,c1,I2,c2,N = 5,size = 64):
	assert(len(I1.shape)==3 and len(I2.shape)==3)
	assert(I1.shape[2] == I2.shape[2])
	assert(c2 <= c1)
	assert(I1.dtype == I2.dtype)
	res_c = c1 + I2.shape[1] - c2
	res_r = max(I1.shape[0], I2.shape[0])

	pad_r , pad_c = 0 , 0
	if res_r % (2**N) != 0:
		pad_r = (2**N)*((res_r + (2**N) - 1) // (2**N)) - res_r
	if res_c % (2**N) != 0:
		pad_c = (2**N)*((res_c + (2**N) - 1) // (2**N)) - res_c
	
	res_r , res_c = res_r + pad_r, res_c + pad_c

	dst1 = np.zeros((res_r,res_c,I1.shape[2]))
	dst1[:I1.shape[0],:I1.shape[1],:] = I1
	LI1 = laplacianPyramid(dst1,N)

	dst2 = np.zeros((res_r,res_c,I1.shape[2]))
	dst2[:I2.shape[0],c1-c2:c1-c2+I2.shape[1],:] = I2
	LI2 = laplacianPyramid(dst2,N)

	LI = []
	i = 0
	for l1 , l2 in zip(LI1,LI2):
		LI.append(blend_layer(l1,l2,c1//(2**i),size//(2**i)))
		i += 1

	return reconstruct(LI,N).astype(I1.dtype)

I1 = cv.imread(os.path.join('PartB','mosaicking','house','0001.jpg'))
I2 = cv.imread(os.path.join('PartB','mosaicking','house','0002.jpg'))
I_ = blend(I1,400,I2,100)

if not os.path.exists(os.path.join('PartB','mosaicking','house','result')):
        os.mkdir(os.path.join('PartB','mosaicking','house','result'))
cv.imwrite(os.path.join('PartB','mosaicking','house','result','output1.jpg'),I_)

#################################################################


# This function unwarp the image and blend accordingly

def blend_unwarp(I1,pts1,I2,pts2,N=5,size=64):
	assert(len(I1.shape)==3 and len(I2.shape)==3)
	assert(I1.shape[2] == I2.shape[2])
	assert(I1.dtype == I2.dtype)
	assert(len(pts1) == 4 and len(pts2) == 4)

	pts21 = np.array([[size//2, size//2],[I1.shape[1]-size, size//2],[I1.shape[1]-size, I1.shape[0]-size//2],[size//2, I1.shape[0] - size//2]])
	unwarp_mat1 = get_perspective_transform_method_b(pts1,pts21)
	pers_I1 = cv.warpPerspective(I1,unwarp_mat1,(I1.shape[1],I1.shape[0]),flags=cv.INTER_LINEAR).astype(I1.dtype)

	pts22 = np.array([[size, size//2],[I2.shape[1]-size//2, size//2],[I2.shape[1]-size//2, I2.shape[0]-size//2],[size, I2.shape[0] - size//2]])
	unwarp_mat2 = get_perspective_transform_method_b(pts2,pts22)
	pers_I2 = cv.warpPerspective(I2,unwarp_mat2,(I2.shape[1],I1.shape[0]),flags=cv.INTER_LINEAR).astype(I2.dtype)

	return blend(pers_I1,I1.shape[1]-size,pers_I2,size,N,size).astype(I1.dtype)

###############################################################


def get_harris_corner_points(I):
	gray = np.float32(cv.cvtColor(I,cv.COLOR_BGR2GRAY))

	harris = cv.cornerHarris(gray,5,5,0.04)
	dil_size = 3
	dil_elem = cv.getStructuringElement(cv.MORPH_RECT, (2 * dil_size + 1, 2 * dil_size + 1),(dil_size, dil_size))
	harris = cv.dilate(harris,None)

	dst = np.zeros(gray.shape).astype(gray.dtype)
	dst[harris > 0.01 * harris.max()] = 255
	dst = np.uint8(dst)
	# from datetime import datetime
	# date = datetime.now().strftime("%m-%d-%Y:%H:%M:%S.%f")
	# cv.imwrite(f"harris{date}.jpg",dst)

	_, _, _, corners = cv.connectedComponentsWithStats(dst)

	return corners.astype('int')

def image_matching(I1,pt1, I2, pt2, pad = 2):
	if pt1[0] < pad or pt1[1] < pad or pt2[0] < pad or pt2[1] < pad:
		return 0
	if pt1[0] > I1.shape[1] - pad - 1 or pt1[1] > I1.shape[0] - pad - 1 or pt2[0] > I2.shape[1] - pad - 1 or pt2[1] > I2.shape[0] - pad - 1:
		return 0

	roi = I1[pt1[1]-pad:pt1[1]+pad+1 , pt1[0]-pad:pt1[0]+pad+1]
	tar = I2[pt2[1]-pad:pt2[1]+pad+1, pt2[0]-pad:pt2[0]+pad+1]
	
	# normalised cross correlation matching
	# corr = np.sum(roi*tar)
	# norm = np.sqrt(np.sum(roi**2)) * np.sqrt(np.sum(tar**2))
	# return corr / (1e-5 + norm)

	# SSD matching
	return np.sum(roi*tar) / np.sum((tar - roi)**2)


def get_matching_points(I1,pts1, I2, pts2):
	ans = []
	for pt1 in pts1:
		max_corr = 0
		matched_pt = None
		for pt2 in pts2:
			n_corr = image_matching(I1,pt1,I2,pt2,5)
			if n_corr > max_corr:
				max_corr = n_corr
				matched_pt = pt2

		if max_corr > 0 :
			ans.append((pt1,matched_pt,max_corr))
			pts2 = np.delete(pts2, np.where(pts2 == matched_pt), axis = 0)

	# print(ans)
	max_score = max(ans, key = lambda i : i[2])[2]
	final_pts1 = [pt for pt , _ , score in ans if score > 0.9 * max_score]
	final_pts2 = [pt for _ , pt , score in ans if score > 0.9 * max_score]

	return np.array(final_pts1) , np.array(final_pts2)

# This function automatically fins matching points of the the images and blend accordingly
def blend_auto(I1,I2,N=5,size=64):
	assert(len(I1.shape)==3 and len(I2.shape)==3)
	assert(I1.shape[2] == I2.shape[2])
	assert(I1.dtype == I2.dtype)

	pts1 = get_harris_corner_points(I1)
	pts2 = get_harris_corner_points(I2)

	pts1 , pts2 = get_matching_points(I1,pts1,I2,pts2)
	c = int(0.5 * (I1.shape[1] + I2.shape[1]))

	res_c = int(0.85 * (I1.shape[1] + I2.shape[1]))
	res_r = max(I1.shape[0], I2.shape[0])

	dst1 = np.zeros((res_r,res_c,I1.shape[2])).astype(I1.dtype)
	dst1[:I1.shape[0],:I1.shape[1],:] = I1

	dst2 = np.zeros((res_r,res_c,I1.shape[2])).astype(I1.dtype)
	dst2[:I2.shape[0],res_c - I2.shape[1]:res_c,:] = I2

	pts2[:,0] = pts2[:,0] + res_c - I2.shape[1]

	unwarp_mat1 = get_perspective_transform_method_b(pts1,pts2)
	pers_I1 = cv.warpPerspective(dst1,unwarp_mat1,(res_c,res_r),flags=cv.INTER_LINEAR).astype(I1.dtype)
	# cv.imwrite("left_pers.jpg",pers_I1)

	unwarp_mat2 = get_perspective_transform_method_b(pts2,pts1)
	pers_I2 = cv.warpPerspective(dst2,unwarp_mat2,(res_c,res_r),flags=cv.INTER_LINEAR).astype(I1.dtype)
	# cv.imwrite("right_pers.jpg",pers_I2)

	# from matplotlib import pyplot as plt
	# # Display original image
	# plt.scatter(x=pts1[:, 0], y=pts1[:, 1], c='r', s=5)
	# plt.imshow(dst1)
	# plt.show()
	# plt.clf()

	# # Display affine image
	# plt.scatter(x=pts2[:, 0], y=pts2[:, 1], c='r', s=5)
	# plt.imshow(dst2)
	# plt.show()
	# plt.clf()

	return blend(pers_I1,c,pers_I2,c,N,size).astype(I1.dtype)

###############################################################

## Test Part-B

In [7]:

if not os.path.exists(os.path.join('PartB','convolve')):
	os.mkdir(os.path.join('PartB','convolve'))
if not os.path.exists(os.path.join('PartB','reduce')):
	os.mkdir(os.path.join('PartB','reduce'))
if not os.path.exists(os.path.join('PartB','expand')):
	os.mkdir(os.path.join('PartB','expand'))
if not os.path.exists(os.path.join('PartB','guass_pyramid')):
	os.mkdir(os.path.join('PartB','guass_pyramid'))
if not os.path.exists(os.path.join('PartB','laplace_pyramid')):
	os.mkdir(os.path.join('PartB','laplace_pyramid'))
if not os.path.exists(os.path.join('PartB','laplace_reconstruct')):
	os.mkdir(os.path.join('PartB','laplace_reconstruct'))

img_file = os.path.join('PartB','lena.png')

# test convolve	
K = (1/16)*np.array([[1,2,1],[2,4,2],[1,2,1]])
I = cv.imread(img_file)
conv = convolve(I,K)
conv_inbuilt = cv.filter2D(I,-1,K)
print("Error in convolve guassian filter : ",calculate_error(conv,conv_inbuilt))
cv.imwrite(os.path.join('PartB','convolve','convolve_guss.jpg'),conv)
print("Convolve applied with guassian kernel and output image saved.")

K = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
I = cv.imread(img_file,0)
conv = convolve(I,K)
conv_inbuilt = cv.filter2D(I,-1,K)
print("Error in convolve vertical edge kernel : ",calculate_error(conv,conv_inbuilt))
cv.imwrite(os.path.join('PartB','convolve','convolve_vedge.jpg'),conv)
print("Convolve applied with vertical edge kernel and output image saved.")


# test reduce
I = cv.imread(img_file)
I_ = reduce(I)
red_inbuilt = cv.resize(I,(0,0),fx=0.5,fy=0.5)
print("Error in reduce : ",calculate_error(I_,red_inbuilt))
cv.imwrite(os.path.join('PartB','reduce','reduce.jpg'),I_)
print("Reduce applied on lena.jpg and output image saved.")

# test expand
I = cv.imread(img_file)
I_ = expand(I)
exp_inbuilt = cv.resize(I,(0,0),fx=2,fy=2)
print("Error in expand : ",calculate_error(I_,exp_inbuilt))
cv.imwrite(os.path.join('PartB','expand','expand.jpg'),I_)
print("Expand applied on lena.jpg and output image saved.")


# test guassian pyramid
I = cv.imread(img_file)
for n in range(1,5):
	I_ = gaussianPyramid(I,n)
	cv.imwrite(os.path.join('PartB','guass_pyramid',f"guass_pyramid_lvl{n}.jpg"),I_)
	print(f"Guassian pyramid on level {n} applied on lena.jpg and output image saved.")

# test laplacian pyramid
I = cv.imread(img_file)
N = 5
LI = laplacianPyramid(I,N)
for n in range(1,len(LI)):
	I_ = LI[n-1]
	cv.imwrite(os.path.join('PartB','laplace_pyramid',f"laplace_pyramid_lvl{n}.jpg"),I_)
print(f"Laplacian Pyramid pyramid on level {N} applied on lena.jpg and output images saved.")

# test reconstruction
N = 5
I = cv.imread(img_file)
LI = laplacianPyramid(I,N)
I_ = reconstruct(LI,N)
cv.imwrite(os.path.join('PartB','laplace_reconstruct','laplace_reconstructed.jpg'),I_)
print(f"Laplacian Pyramid pyramid on level {N} reconstructed and output images saved.")
error = calculate_error(I,I_)
print("MSE error after reconstruction = ",error,"\n")

Error in convolve guassian filter :  0.536474863688151
Convolve applied with guassian kernel and output image saved.
Error in convolve vertical edge kernel :  37.084686279296875
Convolve applied with vertical edge kernel and output image saved.
Error in reduce :  55.07914733886719
Reduce applied on lena.jpg and output image saved.
Error in expand :  13.45495859781901
Expand applied on lena.jpg and output image saved.
Guassian pyramid on level 1 applied on lena.jpg and output image saved.
Guassian pyramid on level 2 applied on lena.jpg and output image saved.
Guassian pyramid on level 3 applied on lena.jpg and output image saved.
Guassian pyramid on level 4 applied on lena.jpg and output image saved.
Laplacian Pyramid pyramid on level 5 applied on lena.jpg and output images saved.
Laplacian Pyramid pyramid on level 5 reconstructed and output images saved.
MSE error after reconstruction =  0.0 

